In [1]:
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!sudo sh -c 'echo "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/cuda.list'
!sudo apt-get update && sudo apt-get install -y --no-install-recommends cuda-drivers

Executing: gpg --ignore-time-conflict --no-options --no-default-keyring --homedir /tmp/tmp.D1fI4BQhih --no-auto-check-trustdb --trust-model always --keyring /etc/apt/trusted.gpg --primary-keyring /etc/apt/trusted.gpg --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
gpg: key 7FA2AF80: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Ign http://archive.ubuntu.com trusty InRelease
Ign http://developer.download.nvidia.com  InRelease
Get:1 http://developer.download.nvidia.com  Release.gpg [819 B]
Get:2 http://developer.download.nvidia.com  Release [564 B]
Get:3 http://archive.ubuntu.com trusty-updates InRelease [65.9 kB]
Get:4 http://archive.ubuntu.com trusty-security InRelease [65.9 kB]
Get:5 http://developer.download.nvidia.com  Packages [44.7 kB]
Get:6 http://archive.ubuntu.com trusty Release.gpg [933 B]
Get:7 http://archive.ubuntu.com trusty-updates/main Sources [485 kB]
Get:8 

In [2]:
!nvidia-smi

Thu Feb 23 00:07:45 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.26                 Driver Version: 375.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   37C    P8    30W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [5]:
print(np.sqrt(mnist.train.images[0].size))
print(mnist.train.labels[0].size)

28.0
10


In [6]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

In [7]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [8]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [10]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [11]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [12]:
# Model
logits = conv_net(x, weights, biases, keep_prob)

In [13]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [14]:
# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
# Initializing the variables
init = tf. global_variables_initializer()

In [17]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            if batch % 100 == 0:
                print('Epoch {:>2}, Batch {:>3} -'
                      'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                        epoch + 1,
                        batch + 1,
                        loss,
                        valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 61501.2656 Validation Accuracy: 0.101562
Epoch  1, Batch 101 -Loss:  5398.4609 Validation Accuracy: 0.476562
Epoch  1, Batch 201 -Loss:  2674.6934 Validation Accuracy: 0.578125
Epoch  1, Batch 301 -Loss:  2574.2749 Validation Accuracy: 0.585938
Epoch  1, Batch 401 -Loss:  1828.7257 Validation Accuracy: 0.632812
Epoch  2, Batch   1 -Loss:  1358.4874 Validation Accuracy: 0.632812
Epoch  2, Batch 101 -Loss:  1070.9467 Validation Accuracy: 0.640625
Epoch  2, Batch 201 -Loss:   858.4261 Validation Accuracy: 0.644531
Epoch  2, Batch 301 -Loss:   914.5276 Validation Accuracy: 0.648438
Epoch  2, Batch 401 -Loss:   691.6619 Validation Accuracy: 0.636719
Epoch  3, Batch   1 -Loss:  1010.9992 Validation Accuracy: 0.640625
Epoch  3, Batch 101 -Loss:   763.0668 Validation Accuracy: 0.625000
Epoch  3, Batch 201 -Loss:   912.0432 Validation Accuracy: 0.605469
Epoch  3, Batch 301 -Loss:   797.8560 Validation Accuracy: 0.566406
Epoch  3, Batch 401 -Loss:   665.7072 Validation

In [18]:
# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# e.g. (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))
X = tf.constant(x)

In [36]:
def conv2d(input):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    F_W = tf.Variable(tf.truncated_normal([2, 2, 1, 3]))
    F_b = tf.Variable(tf.truncated_normal([3]))
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    return tf.nn.bias_add(tf.nn.conv2d(input, F_W, strides, padding), F_b)

In [44]:
def maxpool(input):
    ksize = [1, 2, 2, 1]
    strides = [1, 2, 2, 1]
    padding = 'SAME'
    return tf.nn.max_pool(input, ksize, strides, padding)

In [45]:
out = conv2d(X)

In [46]:
out = maxpool(X)